In [ ]:
import torch
import cv2
from ultralytics import YOLO

# --- Configuration ---

# Path to your trained model weights (assuming 'best.pt' is saved here)
# **ADJUST THIS PATH** if your run folder name is different (e.g., run_2)
MODEL_PATH = 'weld_quality_detection/run_1/weights/best.pt'

# Determine the device dynamically
if torch.cuda.is_available():
    DEVICE = '0'  # Use GPU 0
    print(f"CUDA device detected: Running on GPU {DEVICE}")
else:
    DEVICE = 'cpu'
    print("No CUDA device detected. Running on CPU (real-time performance may be limited).")

# --- Main Inference Function ---
def run_realtime_detection():
    """
    Loads the trained model and runs real-time object detection on the default webcam.
    """
    try:
        # Load the trained model
        print(f"\nLoading trained model from: {MODEL_PATH}")
        model = YOLO(MODEL_PATH)

        # Open the default webcam (typically index 0)
        # You may need to change 0 to 1 or 2 if you have multiple cameras
        cap = cv2.VideoCapture(0)
        
        if not cap.isOpened():
            print("Error: Could not open webcam. Check if another program is using it or if the index (0) is correct.")
            return

        print("\nStarting real-time detection. Press 'q' to quit.")
        
        # --- Prediction Configuration for Webcam ---
        # The stream=True argument is CRITICAL for real-time video processing.
        # It uses a generator to process frames as they arrive.
        results_generator = model.predict(
            source=0,          # 0 refers to the default webcam
            show=True,         # CRITICAL: Displays the resulting frames with bounding boxes
            conf=0.25,         # Minimum confidence to show a detection
            imgsz=320,         # Use the memory-efficient size from training
            half=True,         # Use FP16 for faster GPU inference
            device=DEVICE,     # Run on GPU if available
            stream=True        # Essential for processing video frames continuously
        )

        # The 'predict' function handles the main loop when 'show=True', 
        # but we must keep the process alive manually if we were using a custom loop. 
        # Since 'show=True' is used, Ultralytics manages the display window and the 'q' key press.
        
        # The generator will run until the window is closed or 'q' is pressed
        for _ in results_generator:
            pass # Keep iterating through results until the stream ends

    except FileNotFoundError:
        print(f"\nError: Model weights not found at {MODEL_PATH}")
        print("Please ensure you have trained the model and the path above is correct.")
    except Exception as e:
        print(f"\nAn unexpected error occurred: {e}")
    finally:
        # Release the webcam capture object
        if 'cap' in locals() and cap.isOpened():
            cap.release()
        # Destroy all OpenCV windows
        cv2.destroyAllWindows()

if __name__ == '__main__':
    run_realtime_detection()
